<a href="https://colab.research.google.com/github/Trevorchenmsu/Facebook-Hateful-Memes-Challenge/blob/main/V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# notebook setting set as GPU
from google.colab import drive
drive.mount('/content/drive/')
import os
home = "/content/drive/MyDrive"
os.chdir(home)
os.getcwd()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


'/content/drive/MyDrive'

In [1]:
# Install specified versions of `torch` and `torchvision`, before installing mmf (causes an issue)
!pip install torch==1.6.0 torchvision==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Install MMF from source. Clone the following repo where mmf does not 
# install default image features, since we will use our own features
!git clone --branch no_feats --config core.symlinks=true https://github.com/rizavelioglu/mmf.git

# install mmf
import os
os.chdir(os.path.join(home, "mmf"))
!pip install --editable .

In [2]:
# upload the zip file under this path
import os
os.chdir(os.path.join(home, "data/"))
!mmf_convert_hm --zip_file="img_ocr.zip"  --password="" --bypass_checksum 1

In [3]:
# check the files under the cach folder
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/annotations

# check the number of images
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/ |wc -l

In [ ]:
# Free up the disk by removing .zip, .tar files
!rm -rf /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img_ocr.zip

In [4]:
# check the files under the cach folder
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/annotations

# # check the number of images
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/ |wc -l

In [5]:
# number of unique images in the dataset
import pandas as pd
import os

annotation_dir = "/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/annotations"
img_dir = "/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/"

# Collect all the annotations (from Phase-2)
train       = pd.read_json(f"{annotation_dir}/train.jsonl", lines=True)
dev_seen    = pd.read_json(f"{annotation_dir}/dev_seen.jsonl", lines=True)
dev_unseen  = pd.read_json(f"{annotation_dir}/dev_unseen.jsonl", lines=True)
test_seen   = pd.read_json(f"{annotation_dir}/test_seen.jsonl", lines=True)
test_unseen = pd.read_json(f"{annotation_dir}/test_unseen.jsonl", lines=True)

# Create 2 sets: 
#   A set of strings, 'a': for all the image names,
#   A set of lists, 'b': for all the image names in dataset, e.g. train, dev_seen, etc.
a = os.listdir(f"{img_dir}")
b = []
for i in [train, dev_seen, dev_unseen, test_seen, test_unseen]:
    b.append(list(i["img"].str.split("/").str.get(1)))

set_mapping = ['train', 'dev_seen', 'dev_unseen', 'test_seen', 'test_unseen']
total_size = 0
print("#of images in: ")
for idx, i in enumerate(b):
    total_size += len(set(i))
    print(f"\t'{set_mapping[idx]}'  \t:", len(set(i)))
else:
    print(f"\nIn total there are {total_size} images,",
          "\nBut the # of images in /img/ directory is: ", len(a))

In [6]:
!ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/ |wc -l

In [7]:
# extract image features， intall packages and repos
import os
os.chdir(home)
!git clone https://gitlab.com/vedanuj/vqa-maskrcnn-benchmark

!pip install ninja yacs cython matplotlib

os.chdir(os.path.join(home, "vqa-maskrcnn-benchmark"))
!rm -rf build
!python setup.py build develop

In [8]:
os.chdir(os.path.join(home, "mmf/tools/scripts/features/"))
out_folder = os.path.join(home, "features_OCR/")

!python extract_features_vmb.py --config_file "https://dl.fbaipublicfiles.com/pythia/detectron_model/detectron_model_x152.yaml" \
                                --model_name "X-152" \
                                --output_folder $out_folder \
                                --image_dir "/root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/images/img/" \
                                --num_features 100 \
                                --start_index 7200 \
                                --end_index 8000 \
                                # --exclude_list "/content/exclude.txt"
                                # --feature_name "fc6" \
                                # --confidence_threshold 0. \

In [9]:
# Unzip the feature data
import os
home = "/content/drive/MyDrive"
os.chdir(home)
!unzip features_ocr.zip -d /content/drive/MyDrive/

In [10]:
# Check how many images we have in total
!ls /content/drive/MyDrive/features_ocr/ | wc -l
# !ls /root/.cache/torch/mmf/data/datasets/hateful_memes/defaults/

In [11]:
import os
os.chdir(home)
# Define where image features are
feats_dir = os.path.join(home, "features_ocr")
# Define where train.jsonl is
train_dir = os.path.join(home, "train_500.jsonl")

!mmf_run config="projects/visual_bert/configs/hateful_memes/from_coco.yaml" \
        model="visual_bert" \
        dataset=hateful_memes \
        run_type=train_val \
        checkpoint.max_to_keep=1 \
        checkpoint.resume_zoo=visual_bert.pretrained.cc.full \
        training.tensorboard=True \
        training.checkpoint_interval=50 \
        training.evaluation_interval=50 \
        training.max_updates=500 \
        training.log_interval=100 \
        dataset_config.hateful_memes.max_features=100 \
        dataset_config.hateful_memes.annotations.train[0]=$train_dir \
        dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_unseen.jsonl \
        dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_unseen.jsonl \
        dataset_config.hateful_memes.features.train[0]=$feats_dir \
        dataset_config.hateful_memes.features.val[0]=$feats_dir \
        dataset_config.hateful_memes.features.test[0]=$feats_dir \
        training.lr_ratio=0.3 \
        training.use_warmup=True \
        training.batch_size=32 \
        optimizer.params.lr=5.0e-05 \
        env.save_dir=./trail1_500 \
        env.tensorboard_logdir=logs/fit/trial1_500 \

In [ ]:
# Visualize losses/accuracy via Tensorboard
# Load the TensorBoard notebook extension
%load_ext tensorboard
%tensorboard --logdir logs/fit

In [ ]:
prediction (test_unseen.jsonl)
import os
os.chdir(home)
# where checkpoint is
ckpt_dir = os.path.join(home, "sub1/best.ckpt")
feats_dir = os.path.join(home, "features/feats_hm")

!mmf_predict config="projects/visual_bert/configs/hateful_memes/defaults.yaml" \
    model="visual_bert" \
    dataset=hateful_memes \
    run_type=test \
    checkpoint.resume_file=$ckpt_dir \
    checkpoint.reset.optimizer=True \
    dataset_config.hateful_memes.annotations.val[0]=hateful_memes/defaults/annotations/dev_unseen.jsonl \
    dataset_config.hateful_memes.annotations.test[0]=hateful_memes/defaults/annotations/test_unseen.jsonl \
    dataset_config.hateful_memes.features.train[0]=$feats_dir \
    dataset_config.hateful_memes.features.val[0]=$feats_dir \
    dataset_config.hateful_memes.features.test[0]=$feats_dir \

In [ ]:
# Test pretrained model
import matplotlib.pyplot as plt
import requests

from PIL import Image

from mmf.models.mmbt import MMBT

model = MMBT.from_pretrained("mmbt.hateful_memes.images")

In [ ]:
image_url = "https://i.imgur.com/tEcsk5q.jpg" #@param {type:"string"}
text = "look how many people love you" #@param {type: "string"}
output = model.classify(image_url, text)
plt.imshow(Image.open(requests.get(image_url, stream=True).raw))
plt.axis("off")
plt.show()
hateful = "Yes" if output["label"] == 1 else "No"
print("Hateful as per the model?", hateful)
print(f"Model's confidence: {output['confidence'] * 100:.3f}%")